In [15]:
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import zipfile

# Unzip manually downloaded dataset
cremad_zip = "crema.zip"  # Update with actual path if different

if os.path.exists(cremad_zip):
    with zipfile.ZipFile(cremad_zip, "r") as zip_ref:
        zip_ref.extractall("cremad_data")

# Verify extraction
extracted_files = os.listdir("cremad_data")
print("Extracted files:", extracted_files)

# Update dataset path to 'AudioWAV' folder
cremad_path = os.path.join("cremad_data", "AudioWAV")
if not os.path.exists(cremad_path):
    raise FileNotFoundError("AudioWAV folder not found. Please check the extracted files.")

print("Using dataset folder:", cremad_path)

# Check if extracted folder contains audio files
audio_files = [f for f in os.listdir(cremad_path) if f.lower().endswith(('.wav', '.mp3', '.flac'))]
if not audio_files:
    raise ValueError("No audio files found in the dataset folder. Please check the extracted files.")

X, y = [], []

# Emotion mapping from filenames
emotion_map = {"SAD": "Sad", "ANG": "Angry", "DIS": "Disgusted", "FEA": "Fearful", "HAP": "Happy", "NEU": "Neutral"}

# Function to extract MFCC features
def extract_mfcc(file_path, max_pad_len=50):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    pad_width = max_pad_len - mfcc.shape[1]

    if pad_width > 0:
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_pad_len]

    return mfcc

# Process CREMA-D dataset
for file in audio_files:
    file_path = os.path.join(cremad_path, file)
    mfcc_features = extract_mfcc(file_path)
    X.append(mfcc_features)

    # Extract emotion from filename
    parts = file.split('_')
    if len(parts) > 2:
        emotion_label = emotion_map.get(parts[2], "Unknown")
    else:
        emotion_label = "Unknown"
    y.append(emotion_label)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # Convert emotion labels to integers

# Convert to NumPy arrays
X = np.array(X)
X = X.reshape(X.shape[0], 40, 50, 1)  # Reshape for CNN
y = np.array(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(40, 50, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(len(label_encoder.classes_), activation='softmax')  # Multi-class classification
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

# Save the model
model.save("voice_recognition_model.h5")


Extracted files: ['AudioWAV']
Using dataset folder: cremad_data/AudioWAV


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 38, 48, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 19, 24, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 17, 22, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 8, 11, 64)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 5632)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         721,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 740,614 (2.83 MB)

 Trainable params: 740,614 (2.83 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.2941 - loss: 2.6493 - val_accuracy: 0.3868 - val_loss: 1.4904
Epoch 2/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.3936 - loss: 1.4685 - val_accuracy: 0.4097 - val_loss: 1.4138
Epoch 3/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4140 - loss: 1.4029 - val_accuracy: 0.4520 - val_loss: 1.3787
Epoch 4/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4490 - loss: 1.3517 - val_accuracy: 0.4621 - val_loss: 1.3528
Epoch 5/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5014 - loss: 1.2491 - val_accuracy: 0.3976 - val_loss: 1.5115
Epoch 6/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5389 - loss: 1.1626 - val_accuracy: 0.4419 - val_loss: 1.4097
Epoch 7/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5629 - loss: 1.1161 - val_accuracy: 0.4560 - val_loss: 1.3868
Epoch 8/20
373/373 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5835 - loss: 1.0573 - val_accuracy: 0

Test Accuracy: 0.43
